In [7]:
from scipy.linalg import svd, diagsvd
from copy import deepcopy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

In [6]:
df = pd.read_csv('data/ratings_small.csv').drop(['timestamp'], axis=1)
df = df.pivot(index='userId', columns='movieId', values='rating').fillna(2.5)
matrix = df.to_numpy()
df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
2,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,4.0,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
3,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
4,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,4.0,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
5,2.5,2.5,4.0,2.5,2.5,2.5,2.5,2.5,2.5,2.5,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5


In [ ]:
# comando para freezar meus requirements - apenas da env atual
# pip freeze > requirements.txt